In [27]:
import math
import numpy as np
import sys
!{sys.executable} -m pip3 install matplotlib
import matplotlib.pyplot as mpl

/Applications/Xcode.app/Contents/Developer/usr/bin/python3: No module named pip3


ModuleNotFoundError: No module named 'matplotlib'

In [4]:
#Actuator: (Magnetotorquer)

class magneticTorquer():


    def __init__(self, name, mass, powerConsumption, torqueOutput, totalPowerExpenditure, orientation):
        self.name = name
        self.mass = mass
        self.powerConsumption = powerConsumption
        self.torqueOutput = torqueOutput
        self.relativePosition = [0,"x"]
        self.totalPowerExpenditure = totalPowerExpenditure
        self.orientation = orientation
    

    def getName(self):
        return self.__name
    def setName(self, newName):
        self.__name = newName

    def getMass(self):
        return self.__mass #7.5 kg?
    def setMass(self, newMass):
        self.__mass = newMass
    
    def getPowerConsumption(self):
        return self.__powerConsumption 
    def setPowerConsumption(self, newPowerConsumption):
        self.__powerConsumption = newPowerConsumption
    
    def getTorqueOutput(self):
        return self.__torqueOutput
    def setTorqueOutput(self, newTorqueOutput):
        self.__torqueOutput = self.newTorqueOutput
    
    def getRelativePosition(self): #ranges from (-32.5,32.5) m, x y, (distance from the center, axis)
        return self.__relativePosition
    def getDistanceOrigin(self):
        return self.relativePosition[0]
    def getPositionAxis(self):
        return str(self.relativePosition[1])
    def setRelativePosition(self, distance, axis):
        if axis not in {"x", "y"}:
            raise AssertionError
        self.__relativePosition = np.array([np.clip(distance, -32.5, 32.5),axis])
    def setRelativePosition(self):
        self.relativePosition = [0,"x"]
    def setRelativePosition(self, newRelativePosition):
        self.__relativePosition = self.newRelativePosition
    def setDistanceOrigin(self, distance):
        self.__relativePosition[0] = np.array([np.clip(distance, -32.5, 32.5),self.__relativePosition[1]])
    def setPositionAxis(self, axis):
        if axis not in {"x", "y"}:
            raise AssertionError
        self.__relativePosition[0] = np.array(self.__relativePosition[0], axis)

    def getTotalPowerExpenditure(self):
        return self.__totalPowerExpenditure
    
    def getOrientation(self): #the name of the axis it is on (+y/-y)
        return self.__orientation

In [9]:
# -*- coding: utf-8 -*-
"""Calculations_Disturbance_Torques.ipynb
Original file is located at
    https://colab.research.google.com/drive/1b3scBauWIFCO3S7pDyM5HyxRLg6FP0ot
"""

import math
# Disturbance Torques Calculation

#Solar Radiation Torque
def calculateSolarRad(J,c,A,I,q):
  # in Newtons
  return (J*A*math.cos(I)*(1+q))/c

J = 1367 #Js
c = 3*10**8
As = 65*65 #total surface area
I = 0 #angle of incidence of the solar radiation -> max of cos(I) is 1
q = 0.6 #reflectance factor
solarRadiationTorque = calculateSolarRad(J,c,As,I,q)
print(solarRadiationTorque) # In Newtons

#Magnetic Field Torque
def calculateMagField(B, c, R):
  return float(B)*float(c)/float(R)**3

B = 10000#magnetic field strength in teslas
c = 1 #approximation constant 1-> polar orbit 2-> equatorial orbit
R = 24622000#distance to Neptune's dipole center in meters
#D is satellite's dipole (minimize by including twist in cables)
magneticFieldTorque = calculateMagField(B,c,R)
print(magneticFieldTorque) # In Newtons

#Gravity Gradient Torque
def calculateGravityGradient(mu, R, Iz, Iy, theta):
  return (3.0*float(mu**2)*(float(Iz)-float(Iy))*math.sin(2.0*theta))/(2*float(R**3))

Iz= (1/12)*60*(65^2*65^2) #moments of inertia, assume it is a rectangular plane (1/12)*m*(a^2+b^2)
Iy= (1/12)*60*65*65 #I= (1/12)*m*(b^2)
theta = math.pi/2#max angle the local vertical makes from the z-axis
mu = (6.67*10**-11)*(1.024*10**26) #planetary constant, -G*mass of planet
R = 24622000 #distance to Neptune's dipole center in meters
gravityGradient = calculateGravityGradient(mu, R, Iz, Iy, theta)
print(-gravityGradient)

SumDisturbanceTorques = magneticFieldTorque+gravityGradient+solarRadiationTorque
print(SumDisturbanceTorques)

#Required Torque is any of the Torques*1.25 or Slew Torque
def calculateSlewTorque(theta, I, t):
  return 4*theta*I*(t**2)

I= (1/12)*60*65*65 
#slew angle
#t = time where slew maneuvers must be performed

Td = SumDisturbanceTorques *1.25

#Required Momentum Storage
def calcMomentumStorage(Td, P):
  return Td*(P**4)

# Td Slew Torque
# P Period of the Solar Sail's Orbit

#Calculation of Aerodynamic Torque
#assume it is negligible because not much air in space
def calculateAeroTorque(p, C, A, V, Cg, Cp):
  return 0.5*(p*C*A*V^2)*(Cp-Cg)

# p = atmosphere density
# Cd = #drag coefficient
# A = #area of the front-facing satellite
# V = #spacecraft's velocity
# Cg = #center of gravity
# Cp = #center of aerodynamic pressure


0.030803066666666667
6.699309144330632e-19
0.011573753774530165
0.019229312892136502
